In [ ]:
!pip install torch torchvision pandas scikit-learn tqdm

In [15]:
import torch
from torchvision import transforms, models
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import os
from PIL import Image
from sklearn.preprocessing import LabelEncoder
from sklearn.neural_network import MLPClassifier
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import joblib
from huggingface_hub import snapshot_download
import json
import numpy as np
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [16]:
device

device(type='cuda')

In [4]:
snapshot_download(repo_id="google/spiqa", repo_type="dataset", local_dir='.') ### Mention the local directory path

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 18 files:   0%|          | 0/18 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/2.43k [00:00<?, ?B/s]

SPIQA_train_val_test-A_raw_tex.zip:   0%|          | 0.00/440M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/8.60k [00:00<?, ?B/s]

...

testA_data_viewer.json:   0%|          | 0.00/2.89M [00:00<?, ?B/s]

SPIQA_train.json:   0%|          | 0.00/209M [00:00<?, ?B/s]

SPIQA_train_val_Images.zip:   0%|          | 0.00/32.0G [00:00<?, ?B/s]

train_val%2FSPIQA_val.json:   0%|          | 0.00/1.68M [00:00<?, ?B/s]

'/content'

In [ ]:
!unzip /content/test-A/SPIQA_testA_Images.zip

In [ ]:
!unzip /content/train_val/SPIQA_train_val_Images.zip | head -n 500

In [7]:
file_path = "/content/train_val/SPIQA_train.json"

with open(file_path, 'r') as file:
    text = json.load(file)

data = []
cols = ['paper', 'question', 'answer', 'reference_figure', 'reference_figure_caption']
for paper in text.keys():
    for question in text[paper]['qa']:
        data.append([paper, question['question'], question['answer'], question['reference'], text[paper]['all_figures'][question['reference']]['caption']])

df = pd.DataFrame(data, columns=cols)

file_path = "/content/test-A/SPIQA_testA.json"
with open(file_path, 'r') as file:
    text = json.load(file)

data = []
cols = ['paper', 'question', 'answer', 'reference_figure', 'reference_figure_caption']
for paper in text.keys():
    for question in text[paper]['qa']:
        data.append([paper, question['question'], question['answer'], question['reference'], text[paper]['all_figures'][question['reference']]['caption']])


test_df = pd.DataFrame(data, columns=cols)
test_df['generated_answer'] = np.nan

In [8]:
df.head()

,paper,question,answer,reference_figure,reference_figure_caption
0,1612.01810v3,"Which method converges faster, joint or separa...",The joint method converges faster than the sep...,1612.01810v3-Figure6-1.png,Comparison of the convergence rate between jo...
1,1612.01810v3,What is the effect of increasing the number of...,The boundary recall generally increases and th...,1612.01810v3-Table1-1.png,Boundary recall and time cost comparisons bet...
2,1612.01810v3,What is the main difference between the search...,"The main difference is that in SLIC, each seed...",1612.01810v3-Figure1-1.png,(a) The search method used in SLIC. Each seed...
3,1612.01810v3,How does the number of superpixels affect the ...,"As the number of superpixels decreases, the re...",1612.01810v3-Figure8-1.png,Images segmented by our proposed approach wit...
4,1612.01810v3,Which algorithm produces the most compact sup...,Our approach.,1612.01810v3-Figure7-1.png,Visual comparison of superpixel segmentation ...


In [9]:
len(df)

262524

In [10]:
print(len(test_df))
test_df.head()

666


,paper,question,answer,reference_figure,reference_figure_caption,generated_answer
0,1611.04684v1,What are the main differences between the educ...,The Bonaparte school focuses on outdoor physic...,1611.04684v1-Table1-1.png,A difficult example from QA,NaN
1,1611.04684v1,Which model performs the best for response sel...,The KEHNN model performs the best for response...,1611.04684v1-Table4-1.png,Table 4: Evaluation results on response selection,NaN
2,1611.04684v1,Which model performs best on the Ubuntu datase...,KEHNN,1611.04684v1-Table5-1.png,Accuracy on different length of text,NaN
3,1611.04684v1,What is the role of the knowledge gates in the...,The knowledge gates are responsible for select...,1611.04684v1-Figure1-1.png,Architecture of KEHNN,NaN
4,1611.04684v1,How does the average number of answers per que...,The training set has a higher average number o...,1611.04684v1-Table2-1.png,Table 2: Statistics of the answer selection da...,NaN


In [11]:
df["image_path"] = "/content/SPIQA_train_val_Images/" + df["paper"] + "/" + df["reference_figure"]
test_df["image_path"] = "/content/SPIQA_testA_Images/" + test_df["paper"] + "/" + test_df["reference_figure"]

df['exists'] = df['image_path'].apply(lambda x: os.path.exists(x))
df=df[df['exists']]
len(df)

446

In [14]:
resnet50 = models.resnet50(pretrained=True)
resnet50 = torch.nn.Sequential(*list(resnet50.children())[:-1])
resnet50.eval().to(device)

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [17]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt2 = GPT2LMHeadModel.from_pretrained("gpt2").to(device)
gpt2.eval()

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [26]:
class ImageDataset(Dataset):
    def __init__(self, df, transform):
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = self.df.iloc[idx]['image_path']
        try:
            image = Image.open(img_path).convert('RGB')
        except Exception as e:
            image = Image.new('RGB', (224, 224), (0, 0, 0))  # Placeholder image if error

        image = self.transform(image)
        img_feature = resnet50(image.unsqueeze(0).to(device)).squeeze().to(device)

        return img_feature

In [22]:
dataset = ImageDataset(test_df, transform)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

In [31]:
from transformers import logging
logging.set_verbosity_error()


In [32]:
projection_layer = nn.Linear(2048, 768).to(device)  # Adjust if needed for resnet50's output

# Generate captions
def generate_caption(image_feature):
    with torch.no_grad():
        projected_feature = projection_layer(image_feature).unsqueeze(0)  # Add batch dimension
        input_embeds = projected_feature.unsqueeze(1)  # GPT-2 expects [batch, seq_len, hidden_dim]

        output_ids = gpt2.generate(inputs_embeds=input_embeds, max_length=20)
        caption = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    return caption


In [33]:
answers = []
for img_feature in tqdm(dataloader, desc="Generating Captions", total=len(dataloader)):
    caption = generate_caption(img_feature[0].to(device))
    # print(f"Generated Caption: {caption}")
    answers.append(caption)

Generating Captions: 100%|██████████| 666/666 [02:34<00:00,  4.32it/s]


In [34]:
test_df.head()

,paper,question,answer,reference_figure,reference_figure_caption,generated_answer,image_path
0,1611.04684v1,What are the main differences between the educ...,The Bonaparte school focuses on outdoor physic...,1611.04684v1-Table1-1.png,A difficult example from QA,NaN,/content/SPIQA_testA_Images/1611.04684v1/1611....
1,1611.04684v1,Which model performs the best for response sel...,The KEHNN model performs the best for response...,1611.04684v1-Table4-1.png,Table 4: Evaluation results on response selection,NaN,/content/SPIQA_testA_Images/1611.04684v1/1611....
2,1611.04684v1,Which model performs best on the Ubuntu datase...,KEHNN,1611.04684v1-Table5-1.png,Accuracy on different length of text,NaN,/content/SPIQA_testA_Images/1611.04684v1/1611....
3,1611.04684v1,What is the role of the knowledge gates in the...,The knowledge gates are responsible for select...,1611.04684v1-Figure1-1.png,Architecture of KEHNN,NaN,/content/SPIQA_testA_Images/1611.04684v1/1611....
4,1611.04684v1,How does the average number of answers per que...,The training set has a higher average number o...,1611.04684v1-Table2-1.png,Table 2: Statistics of the answer selection da...,NaN,/content/SPIQA_testA_Images/1611.04684v1/1611....


In [35]:
test_df['generated_answer'] = answers

In [36]:
test_df.head()

,paper,question,answer,reference_figure,reference_figure_caption,generated_answer,image_path
0,1611.04684v1,What are the main differences between the educ...,The Bonaparte school focuses on outdoor physic...,1611.04684v1-Table1-1.png,A difficult example from QA,", the U.S. Supreme Court has ruled that the go...",/content/SPIQA_testA_Images/1611.04684v1/1611....
1,1611.04684v1,Which model performs the best for response sel...,The KEHNN model performs the best for response...,1611.04684v1-Table4-1.png,Table 4: Evaluation results on response selection,".\n\n""I'm not going to be a part of that,"" he ...",/content/SPIQA_testA_Images/1611.04684v1/1611....
2,1611.04684v1,Which model performs best on the Ubuntu datase...,KEHNN,1611.04684v1-Table5-1.png,Accuracy on different length of text,".\n\n""The government has been very clear that ...",/content/SPIQA_testA_Images/1611.04684v1/1611....
3,1611.04684v1,What is the role of the knowledge gates in the...,The knowledge gates are responsible for select...,1611.04684v1-Figure1-1.png,Architecture of KEHNN,", and the court shall order the defendant to p...",/content/SPIQA_testA_Images/1611.04684v1/1611....
4,1611.04684v1,How does the average number of answers per que...,The training set has a higher average number o...,1611.04684v1-Table2-1.png,Table 2: Statistics of the answer selection da...,", the U.S. government has been using the ""war ...",/content/SPIQA_testA_Images/1611.04684v1/1611....


In [37]:
test_df.drop(columns=['image_path'], inplace=True)
test_df.to_csv('resnet_mlp_results.csv', index=False)